This exercise will require you to pull some data from the Quandl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '5MvsePF6gSHeUSeFCvxU'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections

In [3]:
api_endpoint = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json'

In [4]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
response = requests.get(api_endpoint, params={'api_key': API_KEY, 'start_date': '2020-04-22', 'end_date': '2020-04-22'})

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json_data = response.json()
json_data

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-04-08) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-04-22T22:16:31.570Z',
  'newest_available_date': '2020-04-22',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2020-04-22',
  'end_date': '2020-04-22',
  'data': [['2020-04-22',
    92.75,
    94.65,
    91.6,
    94.65,
    None,
    126830.0,
    11898048.5,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [6]:
response = requests.get(api_endpoint, params={'api_key': API_KEY, 'start_date': '2017-01-01', 'end_date': '2017-12-31'})

2. Convert the returned JSON object into a Python dictionary.

In [7]:
json_data = response.json()
print(type(json_data))

<class 'dict'>


In [8]:
column_names = [name.replace(' ', '_').lower() for name in json_data['dataset']['column_names']]
DailyData = collections.namedtuple('DailyData', column_names)
# Creating a list of named tuples and ordering them in ascending date order
daily_data = [DailyData(*data) for data in json_data['dataset']['data']][::-1]

3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [9]:
open_low = min([data.open for data in daily_data if data.open])
open_high = max([data.open for data in daily_data if data.open])
print("Highest Opening Price for AFX_X on FSE in 2017: ", open_high)
print("Lowest Opening Price for AFX_X on FSE in 2017: ", open_low)

Highest Opening Price for AFX_X on FSE in 2017:  53.11
Lowest Opening Price for AFX_X on FSE in 2017:  34.0


4. What was the largest change in any one day (based on High and Low price)?

In [10]:
change_high = max([d.high - d.low for d in daily_data])
print("Largest Single Day Price Change (High - Low) for AFX_X on FSE in 2017: ", round(change_high, 2))

Largest Single Day Price Change (High - Low) for AFX_X on FSE in 2017:  2.81


5. What was the largest change between any two days (based on Closing Price)?

In [11]:
change_closing = [daily_data[n+1].close - daily_data[n].close for n in range(len(daily_data) - 1)]
if (abs(max(change_closing)) >= abs(min(change_closing))):
    largest_change = max(change_closing)
else:
    largest_change = min(change_closing)

print("Largest Change in Closing Price on Subsequent Days for AFX_X on FSE in 2017: ", round(largest_change, 2))

Largest Change in Closing Price on Subsequent Days for AFX_X on FSE in 2017:  -2.56


6. What was the average daily trading volume during this year?

In [12]:
from functools import reduce
traded_volume = [data.traded_volume for data in daily_data]
avg_traded_volume = reduce(lambda a, b: a + b, traded_volume)/len(traded_volume)

print("Average Trading Volume for AFX_X on FSE in 2017: ", round(avg_traded_volume, 2))

Average Trading Volume for AFX_X on FSE in 2017:  89124.34


7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [13]:
sorted_traded_volume = sorted(traded_volume)
count = len(sorted_traded_volume)
if count % 2 == 0:
    median_traded_volume = (sorted_traded_volume[count // 2 - 1] + sorted_traded_volume[count // 2])/2
else:
    median_traded_volume = sorted_traded_volume[count // 2]

print("Median Trading Volume for AFX_X on FSE in 2017: ", round(median_traded_volume, 2))

Median Trading Volume for AFX_X on FSE in 2017:  76286.0
